In [14]:
import sympy as sym
from scipy.optimize import minimize
import numpy as np
import plotly.graph_objects as go

In [15]:
class Task:
    def __init__(self, a, b, c, d, k, l, m, n):
        self.q1 = "a + b * p_1"
        self.q2 = "c + d * p_2"
        self.C = "k * q_1 ** 2 + l * q_1 * q_2 + m * q_2 ** 2 + n"
        self.P = "p_1 * q_1 + p_2 * q_2 - C"
        self.sym_q1 = sym.parse_expr(self.q1).subs({'a': a, 'b': b})
        self.sym_q2 = sym.parse_expr(self.q2).subs({'c': c, 'd': d})
        self.sym_C = sym.parse_expr(self.C).subs({'k': k, 'l':l, 'm':m, 'n':n, 'q_1':self.sym_q1, 'q_2': self.sym_q2})
        self.sym_P = sym.parse_expr(self.P).subs({'q_1': self.sym_q1, 'q_2': self.sym_q2, 'C': self.sym_C})
        self.lambdify_p = lambda x: sym.lambdify(['p_1', 'p_2'], x)
        self.max_p1, \
        self.max_p2, \
        self.max_P = self.find_max()
        self.max_q1 = self.lambdify_p(self.sym_q1)(self.max_p1, self.max_p2)
        self.max_q2 = self.lambdify_p(self.sym_q2)(self.max_p1, self.max_p2)
        self.max_C = self.lambdify_p(self.sym_C)(self.max_p1, self.max_p2)
        self.plot_P_f = sym.parse_expr(self.P)\
                            .subs(dict(C=self.C))\
                            .subs(dict(k=k, l=l, m=m, n=n))\
                            .subs(dict(p_1=self.max_p1, p_2=self.max_p2))
        self.plots_data = {
            'x': np.linspace(1, 100, 100),
            'y': np.linspace(1, 100, 100),
        }
        self.plots_data['z'] = np.vectorize(sym.lambdify(['q_1', 'q_2'], self.plot_P_f))(*np.meshgrid(self.plots_data['x'], self.plots_data['y']))
        self.surface_plot, \
        self.contour_plot = self._get_plots()



    def _get_plots(self):
        scene = go.Scene(
        xaxis = {'title': 'q1 - спрос на первый товар'},
        yaxis = {'title': 'q2 - спрос на второй товар'},
        zaxis = {'title': 'прибыль'})

        layout = go.Layout(scene=scene)

        return self._get_surface_plot(layout), \
               self._get_contour_plot(layout)


    def _get_surface_plot (self, layout):
        surface = go.Surface(**self.plots_data)
        scatter_3d = go.Scatter3d(
                          x=[self.max_q1],
                          y=[self.max_q2],
                          z=[self.max_P],
                          mode="markers+text",
                          text=[f'M* ({self.max_q1}, {self.max_q2})']
                          )

        fig = go.Figure(data=[surface, scatter_3d], layout=layout)
        return fig


    def _get_contour_plot(self, layout):
        contour = go.Contour(**self.plots_data)
        fig = go.Figure(data=[contour], layout=layout)

        fig.add_trace(go.Scatter(
            x=[self.max_q1],
            y=[self.max_q2],
            mode="markers+text",
            textposition="bottom center",
            text= [f'M* ({self.max_q1},{self.max_q2})']))

        return fig

    def __repr__(self):
        return f"""Оптимальные значения:
p1 = {self.max_p1}
p2 = {self.max_p2}
q1 = {self.max_q1}
q2 = {self.max_q2}
C = {self.max_C}
P = {self.max_P}
"""


    def find_max(self):
        f = self.lambdify_p(self.sym_P)
        res = minimize(lambda x: -f(x[0], x[1]), (0, 0))
        return res.x[0], res.x[1], -res.fun
res = Task(76, -4, 54, -1, 2, 2, 4, 9)
res

c:\users\ace\appdata\local\programs\python\python39\lib\site-packages\plotly\graph_objs\_deprecations.py:489: DeprecationWarning:

plotly.graph_objs.Scene is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Scene




Оптимальные значения:
p1 = 18.499999991669625
p2 = 48.99999997778061
q1 = 2.0000000333214984
q2 = 5.000000022219389
C = 137.0000015774401
P = 145.00000000000003

In [16]:
res.surface_plot

In [17]:
res.contour_plot